In [739]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as BS
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
import json
import numpy as np
from itertools import chain

# Stock Categorization

In [544]:
stocks = pd.read_csv('all_sectors.csv')
stocks = stocks.drop(columns=['Unnamed: 0'])
stocks

,Ticker,Company,Sector
0,FEAM,5E Advanced Materials Inc,Basic Materials
1,ASIX,AdvanSix Inc,Basic Materials
2,AEM,Agnico Eagle Mines Limited,Basic Materials
3,APD,"Air Products and Chemicals, Inc",Basic Materials
4,AGI,Alamos Gold Inc,Basic Materials
...,...,...,...
6104,VIA,Via Renewables Inc - Class A,Utilities
6105,VWTR,Vidler Water Resources Inc,Utilities
6106,VST,Vistra Energy Corp,Utilities
6107,WEC,"WEC Energy Group, Inc",Utilities


In [554]:
basic_materials = list(stocks[stocks.Sector == 'Basic Materials']['Ticker'])
communication_services = list(stocks[stocks.Sector == 'Communication Services']['Ticker'])
consumer_defensive = list(stocks[stocks.Sector == 'Consumer Cyclical']['Ticker'])
consumer_cyclical = list(stocks[stocks.Sector == 'Consumer Defensive']['Ticker'])
energy = list(stocks[stocks.Sector == 'Energy']['Ticker'])
financial_services = list(stocks[stocks.Sector == 'Financial Services']['Ticker'])
healthcare = list(stocks[stocks.Sector == 'Healthcare']['Ticker'])
industrials = list(stocks[stocks.Sector == 'Industrials']['Ticker'])
technology = list(stocks[stocks.Sector == 'Technology']['Ticker'])
utilities = list(stocks[stocks.Sector == 'Utilities']['Ticker'])

In [560]:
sector_dict = {'Basic Materials':basic_materials, 'Communication Services':communication_services, 'Consumer Defensive':consumer_defensive, 'Consumer Cyclical':consumer_cyclical, 'Energy':energy, 'Financial Services':financial_services, 'Healthcare':healthcare, 'Industrials':industrials, 'Technology':technology, 'Utilities':utilities}

In [561]:
sector_dict

{'Basic Materials': ['FEAM',
  'ASIX',
  'AEM',
  'APD',
  'AGI',
  'ALB',
  'AA',
  'AXU',
  'ASTL',
  'AAU',
  'AMR',
  'ALTO',
  'ACH',
  'AREC',
  'AVD',
  'USAS',
  'AMRS',
  'AU',
  'RKDA',
  'MT',
  'ASH',
  'AVTR',
  'AVNT',
  'ASM',
  'AXTA',
  'BTG',
  'BCPC',
  'GOLD',
  'BHIL',
  'BHP',
  'BIOX',
  'BCC',
  'BAK',
  'CBT',
  'CMCL',
  'CE',
  'CPAC',
  'CX',
  'CGAU',
  'CENX',
  'CF',
  'CCF',
  'CGA',
  'CHNR',
  'CXDC',
  'CINR',
  'CLW',
  'CLF',
  'CNEY',
  'CCNC',
  'CDE',
  'CMC',
  'SID',
  'BVN',
  'CMP',
  'CSTM',
  'CTGO',
  'CMT',
  'CTVA',
  'CRH',
  'CRKN',
  'UAN',
  'DNMR',
  'DOW',
  'DRD',
  'DD',
  'EXP',
  'EMN',
  'ECL',
  'ECVT',
  'EGO',
  'ESI',
  'EMX',
  'EXK',
  'EVA',
  'EQX',
  'ERO',
  'EXN',
  'FOE',
  'GSM',
  'AG',
  'FSI',
  'FMC',
  'FSM',
  'FNV',
  'FCX',
  'FRD',
  'FURY',
  'FF',
  'GAU',
  'GATO',
  'GCP',
  'GGB',
  'GEVO',
  'AUMN',
  'GFI',
  'GLDG',
  'GORO',
  'GROY',
  'GSV',
  'GPL',
  'GPRE',
  'SIM',
  'GURE',
  'HMY',
  'HWK

In [563]:
transactions = pd.read_csv(r'transactions/all_transactions (2).csv')

In [577]:
transactions = transactions[(transactions['asset_type'] == 'Stock')|(transactions['asset_type'] == 'Stock Option')]
transactions

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date
0,03/09/2022,Self,TACO,"Del Taco Restaurants, Inc. - Common Stock",Stock,Sale (Partial),"$15,001 - $50,000",Asset held by Roundstone Ventures LLC,"William F Hagerty, Iv",https://efdsearch.senate.gov/search/view/ptr/3...,04/13/2022
15,03/30/2022,Joint,WST,"West Pharmaceutical Services, Inc. Common Stock",Stock,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/0...,04/08/2022
16,03/30/2022,Joint,VEEV,Veeva Systems Inc. Class A Common Stock,Stock,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/0...,04/08/2022
17,03/30/2022,Joint,VMW,"Vmware, Inc. Common stock, Class A",Stock,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/0...,04/08/2022
18,03/30/2022,Joint,SNA,Snap-On Incorporated Common Stock,Stock,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/0...,04/08/2022
...,...,...,...,...,...,...,...,...,...,...,...
8406,12/11/2014,Spouse,DVFAX,Cohen &amp; Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015
8412,12/12/2014,Spouse,XOM,Exxon Mobil Corporation (NYSE),Stock,Sale (Full),"$1,001 - $15,000",Sep,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015
8413,12/19/2014,Spouse,BA,The Boeing Company (NYSE),Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015
8414,12/12/2014,Spouse,BAC,Bank of America Corporation (NYSE),Stock,Sale (Full),"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015


In [594]:
for index,value in tqdm(transactions['senator'].iteritems()):
    if value == 'William F Hagerty, Iv':
        transactions.loc[index, 'senator_rev'] = 'Hagerty, Bill'
    if value == 'Thomas H Tuberville':
        transactions.loc[index, 'senator_rev'] = 'Tuberville, Tommy'
    if value == 'Thomas R Carper':
        transactions.loc[index, 'senator_rev'] = 'Carper, Thomas'
    if value == 'Ron L Wyden':
        transactions.loc[index, 'senator_rev'] = 'Wyden, Ron'
    if value == 'A. Mitchell Mcconnell, Jr.':
        transactions.loc[index, 'senator_rev'] = 'McConnell, Mitch'
    if value == 'Shelley M Capito':
        transactions.loc[index, 'senator_rev'] = 'Capito, Shelley'
    if value == 'Patrick J Toomey':
        transactions.loc[index, 'senator_rev'] = 'Toomey, Pat'
    if value == 'Mark R Warner':
        transactions.loc[index, 'senator_rev'] = 'Warner, Mark'
    if value == 'Susan M Collins':
        transactions.loc[index, 'senator_rev'] = 'Collins, Susan'
    if value == 'Jerry Moran,':
        transactions.loc[index, 'senator_rev'] = 'Moran, Jerry'
    if value == 'John R Thune':
        transactions.loc[index, 'senator_rev'] = 'Thune, John'
    if value == 'John W Hickenlooper':
        transactions.loc[index, 'senator_rev'] = 'Hickenlooper, John'
    if value == 'Jacklyn S Rosen':
        transactions.loc[index, 'senator_rev'] = 'Rosen, Jacky'
    if value == 'Gary C Peters':
        transactions.loc[index, 'senator_rev'] = 'Peters, Gary'
    if value == 'Roy Blunt':
        transactions.loc[index, 'senator_rev'] = 'Blunt, Roy'
    if value == 'Cynthia M Lummis':
        transactions.loc[index, 'senator_rev'] = 'Lummis, Cynthia'
    if value == 'Sheldon Whitehouse':
        transactions.loc[index, 'senator_rev'] = 'Whitehouse, Sheldon'
    if value == 'Rand Paul':
        transactions.loc[index, 'senator_rev'] = 'Paul, Rand'
    if value == 'Angus S King, Jr.':
        transactions.loc[index, 'senator_rev'] = 'King, Angus'
    if value == 'John Boozman':
        transactions.loc[index, 'senator_rev'] = 'Boozman, John'
    if value == 'Ladda Tammy Duckworth':
        transactions.loc[index, 'senator_rev'] = 'Duckworth, Tammy'
    if value == 'Daniel S Sullivan':
        transactions.loc[index, 'senator_rev'] = 'Sullivan, Dan'
    if value == 'James M Inhofe':
        transactions.loc[index, 'senator_rev'] = 'Inhofe, Jim'
    if value == 'Pat Roberts':
        transactions.loc[index, 'senator_rev'] = 'Roberts, Pat'
    if value == 'William Cassidy':
        transactions.loc[index, 'senator_rev'] = 'Cassidy, Bill'
    if value == 'Kelly Loeffler':
        transactions.loc[index, 'senator_rev'] = 'Loeffler, Kelly'
    if value == 'Timothy M Kaine':
        transactions.loc[index, 'senator_rev'] = 'Kaine, Tim'
    if value == 'David A Perdue , Jr':
        transactions.loc[index, 'senator_rev'] = 'Perdue, David'
    if value == 'Tina Smith':
        transactions.loc[index, 'senator_rev'] = 'Smith, Tina'
    if value == 'John Hoeven':
        transactions.loc[index, 'senator_rev'] = 'Hoeven, John'
    if value == 'John N Kennedy':
        transactions.loc[index, 'senator_rev'] = 'Kennedy, John'
    if value == 'Rafael E Cruz':
        transactions.loc[index, 'senator_rev'] = 'Cruz, Ted'
    if value == 'Christopher A Coons':
        transactions.loc[index, 'senator_rev'] = 'Coons, Christopher'
    if value == 'Thomas Udall':
        transactions.loc[index, 'senator_rev'] = 'Udall, Tom'
    if value == 'John F Reed':
        transactions.loc[index, 'senator_rev'] = 'Reed, John'
    if value == 'Thomas R Tillis':
        transactions.loc[index, 'senator_rev'] = 'Tillis, Thom'
    if value == 'Robert P Casey, Jr.':
        transactions.loc[index, 'senator_rev'] = 'Casey, Bob'
    if value == 'Tammy Duckworth':
        transactions.loc[index, 'senator_rev'] = 'Duckworth, Tammy'
    if value == 'Michael F Bennet':
        transactions.loc[index, 'senator_rev'] = 'Bennet, Michael'
    if value == 'Patty Murray':
        transactions.loc[index, 'senator_rev'] = 'Murray, Patty'
    if value == 'Steve Daines':
        transactions.loc[index, 'senator_rev'] = 'Daines, Steve'
    if value == 'Joseph Manchin, Iii':
        transactions.loc[index, 'senator_rev'] = 'Manchin, Joe'
    if value == 'Chris Van Hollen':
        transactions.loc[index, 'senator_rev'] = 'Van Hollen, Chris'
    if value == 'John Cornyn':
        transactions.loc[index, 'senator_rev'] = 'Cornyn, John'
    if value == 'Maria Cantwell':
        transactions.loc[index, 'senator_rev'] = 'Cantwell, Maria'
    if value == 'Michael  B Enzi':
        transactions.loc[index, 'senator_rev'] = 'Enzi, Michael'
    if value == 'Mike Rounds':
        transactions.loc[index, 'senator_rev'] = 'Rounds, Mike'

0it [00:00, ?it/s]

In [712]:
sen_list = ['Hagerty, Bill',
 'Tuberville, Tommy',
 'Carper, Thomas',
 'Wyden, Ron',
 'McConnell, Mitch',
 'Capito, Shelley',
 'Toomey, Pat',
 'Warner, Mark',
 'Collins, Susan',
 'Moran, Jerry',
 'Thune, John',
 'Hickenlooper, John',
 'Rosen, Jacky',
 'Peters, Gary',
 'Blunt, Roy',
 'Lummis, Cynthia',
 'Whitehouse, Sheldon',
 'Paul, Rand',
 'King, Angus',
 'Boozman, John',
 'Duckworth, Tammy',
 'Sullivan, Dan',
 'Inhofe, Jim',
 'Roberts, Pat',
 'Cassidy, Bill',
 'Loeffler, Kelly',
 'Kaine, Tim',
 'Perdue, David',
 'Smith, Tina',
 'Hoeven, John',
 'Kennedy, John',
 'Cruz, Ted',
 'Coons, Christopher',
 'Udall, Tom',
 'Reed, John',
 'Tillis, Thom',
 'Casey, Bob',
 'Bennet, Michael',
 'Murray, Patty',
 'Daines, Steve',
 'Manchin, Joe',
 'Van Hollen, Chris',
 'Cornyn, John',
 'Cantwell, Maria',
 'Enzi, Michael',
 'Rounds, Mike']

In [632]:
sen_stock_dict = {}
for sen in sen_list:
    if sen in sen_stock_dict:
        sen_stock_dict[sen].extend(list(transactions[transactions.senator_rev == sen]['ticker']))
    else:
        sen_stock_dict[sen] = list(transactions[transactions.senator_rev == sen]['ticker'])

In [727]:
amount_series = transactions.amount
for index,value in amount_series.iteritems():
    transactions.loc[index, 'amount'] = value.replace('$','')

In [ ]:
transactions = transactions.drop(columns=['comment','ptr_link','senator'])

In [732]:
transactions = transactions.rename(columns={'asset_description':'Asset Description','transaction_date':'Transaction Date','owner':'Owner','ticker':'Ticker','asset_type':'Asset Type','type':'Type','amount':'Amount','disclosure_date':'Disclosure Date','senator_rev':'Senator'})

In [733]:
transactions

,Transaction Date,Owner,Ticker,Asset Description,Asset Type,Type,Amount,Disclosure Date,Senator
0,03/09/2022,Self,TACO,"Del Taco Restaurants, Inc. - Common Stock",Stock,Sale (Partial),"15,001 - 50,000",04/13/2022,"Hagerty, Bill"
15,03/30/2022,Joint,WST,"West Pharmaceutical Services, Inc. Common Stock",Stock,Sale (Full),"1,001 - 15,000",04/08/2022,"Tuberville, Tommy"
16,03/30/2022,Joint,VEEV,Veeva Systems Inc. Class A Common Stock,Stock,Sale (Full),"1,001 - 15,000",04/08/2022,"Tuberville, Tommy"
17,03/30/2022,Joint,VMW,"Vmware, Inc. Common stock, Class A",Stock,Sale (Full),"1,001 - 15,000",04/08/2022,"Tuberville, Tommy"
18,03/30/2022,Joint,SNA,Snap-On Incorporated Common Stock,Stock,Sale (Full),"1,001 - 15,000",04/08/2022,"Tuberville, Tommy"
...,...,...,...,...,...,...,...,...,...
8406,12/11/2014,Spouse,DVFAX,Cohen &amp; Steers Dividend Value A (NASDAQ),Stock,Purchase,"1,001 - 15,000",01/09/2015,"Collins, Susan"
8412,12/12/2014,Spouse,XOM,Exxon Mobil Corporation (NYSE),Stock,Sale (Full),"1,001 - 15,000",01/05/2015,"Roberts, Pat"
8413,12/19/2014,Spouse,BA,The Boeing Company (NYSE),Stock,Purchase,"1,001 - 15,000",01/05/2015,"Roberts, Pat"
8414,12/12/2014,Spouse,BAC,Bank of America Corporation (NYSE),Stock,Sale (Full),"1,001 - 15,000",01/05/2015,"Roberts, Pat"


# Bill Categorization

URL iteration list

In [982]:
normal_policy_list = ['Health', 'Public Lands and Natural Resources', 'Agriculture and Food', 'Economics and Public Finance', 'Housing and Community Development', 'Water Resources Development']

In [983]:
url_policy_list = list(map(lambda x: x.replace(' ', '+').replace(',', '%2C'), normal_policy_list))

In [ ]:
var_policy_list = list(map(lambda x: x.replace(' ', '_').replace(',', ''), normal_policy_list))

Retreiving all items on one page

response = requests.get('https://www.congress.gov/search?pageSort=title&pageSize=250&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D')

soup = BS(response.text)

soup.find_all('span', attrs={'class':'result-heading'})

bill_tags = str(soup.find_all('span', attrs={'class':'result-heading'}))

#Bills
URL='https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D&pageSize=250&page=1'
response = requests.get(URL)
soup = BS(response.text)
pages = int(re.search('class="results-number"> of (\d+)</span>', str(soup.find_all('span', attrs={'class':'results-number'}))).group(1))
for x in tqdm(range(1,pages+1)):
    response = requests.get(f'https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D&pageSize=250&page={x}')
    soup = BS(response.text)
    bill_tags = str(soup.find_all('span', attrs={'class':'result-heading'}))
    print(x,len(re.findall('r=\d+">(S.\d+|H.R.\d+|S.Res.\d+|H.Res.\d+|S.Con.Res.\d+|H.Con.Res.\d+|S.J.Res.\d+|H.J.Res.\d+)</a>', bill_tags)[1::2]))

#Congress
URL='https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D&pageSize=250&page=1'
response = requests.get(URL)
soup = BS(response.text)
pages = int(re.search('class="results-number"> of (\d+)</span>', str(soup.find_all('span', attrs={'class':'results-number'}))).group(1))
for x in tqdm(range(1,pages+1)):
    response = requests.get(f'https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D&pageSize=250&page={x}')
    soup = BS(response.text)
    bill_tags = str(soup.find_all('span', attrs={'class':'result-heading'}))
    print(x,len(re.findall('</a> — (\d+th Congress \\(\d{4}-\d{4}\\))</span>', bill_tags)[1::2]))

len(re.findall('</a> — (\d+th Congress \\(\d{4}-\d{4}\\))</span>', bill_tags)[1::2])

bill_tags

Method for testing any given policy

#Bills
#must replace both URLs with pertinent policy url
URL='https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D&pageSize=250&page=1'
response = requests.get(URL)
soup = BS(response.text)
pages = int(re.search('class="results-number"> of (\d+)</span>', str(soup.find_all('span', attrs={'class':'results-number'}))).group(1))
bills = []

for x in tqdm(range(1,pages+1)):
            response = requests.get(f'https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D&pageSize=250&page={x}')
            soup = BS(response.text)
            bill_tags = str(soup.find_all('span', attrs={'class':'result-heading'}))
            bills.extend(list(re.findall('r=\d+">(S.\d+|H.R.\d+|S.Res.\d+|H.Res.\d+|S.Con.Res.\d+|H.Con.Res.\d+|S.J.Res.\d+|H.J.Res.\d+)</a>', bill_tags)[1::2]))

#Congress
#must replace both URLs with pertinent policy url
URL='https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D&pageSize=250&page=1'
response = requests.get(URL)
soup = BS(response.text)
pages = int(re.search('class="results-number"> of (\d+)</span>', str(soup.find_all('span', attrs={'class':'results-number'}))).group(1))
congress = []

for x in tqdm(range(1,pages+1)):
            response = requests.get(f'https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22Health%22%7D&pageSize=250&page={x}')
            soup = BS(response.text)
            bill_tags = str(soup.find_all('span', attrs={'class':'result-heading'}))
            congress.extend(list(re.findall('</a> — (\d+th Congress \\(\d{4}-\d{4}\\))</span>', bill_tags)[1::2]))

In [1012]:
d = {}
for policy in tqdm(url_policy_list):
    URL=f'https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22{policy}%22%7D&pageSize=250&page=1'
    response = requests.get(URL)
    soup = BS(response.text)
    pages = int(re.search('class="results-number"> of (\d+)</span>', str(soup.find_all('span', attrs={'class':'results-number'}))).group(1))
    for x in tqdm(range(1,pages+1)):
        response = requests.get(f'https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22{policy}%22%7D&pageSize=250&page={x}')
        soup = BS(response.text)
        bill_tags = str(soup.find_all('span', attrs={'class':'result-heading'}))
        if policy in d:
            d[policy].extend(re.findall('</a> — (\d+th Congress \\(\d{4}-\d{4}\\))</span>', bill_tags)[1::2])
        else:
            d[policy] = re.findall('</a> — (\d+th Congress \\(\d{4}-\d{4}\\))</span>', bill_tags)[1::2]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [1013]:
d2 = {}
for policy in tqdm(url_policy_list):
    URL=f'https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22{policy}%22%7D&pageSize=250&page=1'
    response = requests.get(URL)
    soup = BS(response.text)
    pages = int(re.search('class="results-number"> of (\d+)</span>', str(soup.find_all('span', attrs={'class':'results-number'}))).group(1))
    for x in tqdm(range(1,pages+1)):
        response = requests.get(f'https://www.congress.gov/search?pageSort=title&q=%7B%22congress%22%3A%5B%22117%22%2C%22112%22%2C%22113%22%2C%22114%22%2C%22115%22%2C%22116%22%5D%2C%22source%22%3A%22legislation%22%2C%22subject%22%3A%22{policy}%22%7D&pageSize=250&page={x}')
        soup = BS(response.text)
        bill_tags = str(soup.find_all('span', attrs={'class':'result-heading'}))
        if policy in d2:
            d2[policy].extend(re.findall('r=\d+">(S.\d+|H.R.\d+|S.Res.\d+|H.Res.\d+|S.Con.Res.\d+|H.Con.Res.\d+|S.J.Res.\d+|H.J.Res.\d+)</a>', bill_tags)[1::2])
        else:
            d2[policy] = re.findall('r=\d+">(S.\d+|H.R.\d+|S.Res.\d+|H.Res.\d+|S.Con.Res.\d+|H.Con.Res.\d+|S.J.Res.\d+|H.J.Res.\d+)</a>', bill_tags)[1::2]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [1014]:
for policy in url_policy_list:
    print(policy,len(d[policy]))
    print(policy,len(d2[policy]))

Health 8911
Health 8911
Public+Lands+and+Natural+Resources 3722
Public+Lands+and+Natural+Resources 3722
Agriculture+and+Food 1453
Agriculture+and+Food 1453
Economics+and+Public+Finance 1326
Economics+and+Public+Finance 1326
Housing+and+Community+Development 1005
Housing+and+Community+Development 1005
Water+Resources+Development 552
Water+Resources+Development 552


In [1076]:
for k, v in d.items():
    for x in range(len(d[k])):
        d[k][x] = {d[k][x]:d2[k][x]}

In [1080]:
d

{'Health': [{'114th Congress (2015-2016)': 'H.R.34'},
  {'114th Congress (2015-2016)': 'H.R.6'},
  {'116th Congress (2019-2020)': 'H.R.3558'},
  {'115th Congress (2017-2018)': 'H.R.5806'},
  {'114th Congress (2015-2016)': 'S.674'},
  {'114th Congress (2015-2016)': 'H.R.3652'},
  {'117th Congress (2021-2022)': 'S.2837'},
  {'115th Congress (2017-2018)': 'H.R.5598'},
  {'115th Congress (2017-2018)': 'H.R.4710'},
  {'112th Congress (2011-2012)': 'H.R.2674'},
  {'116th Congress (2019-2020)': 'H.R.1559'},
  {'117th Congress (2021-2022)': 'H.R.7097'},
  {'116th Congress (2019-2020)': 'H.R.8767'},
  {'117th Congress (2021-2022)': 'H.R.1859'},
  {'115th Congress (2017-2018)': 'H.R.3834'},
  {'117th Congress (2021-2022)': 'H.R.4965'},
  {'117th Congress (2021-2022)': 'S.2683'},
  {'117th Congress (2021-2022)': 'H.R.7232'},
  {'117th Congress (2021-2022)': 'H.R.7116'},
  {'116th Congress (2019-2020)': 'H.R.8736'},
  {'114th Congress (2015-2016)': 'S.2423'},
  {'113th Congress (2013-2014)': 'S.80

## Roll Call Srape
=================================================================================

# Pulling data from individual page

Pulling data from senate vote #134

response = requests.get('https://www.govtrack.us/congress/votes/117-2022/s134')

soup = BS(response.text)

print(soup.prettify())

title = soup.find('h1').text.replace('\n  ', '').replace('\n', '')
title

date = soup.find('div', attrs={'style':'margin: 2px 0 16px 0; font-size: 12px; font-weight: normal; color: black; line-height: 125%; padding-bottom: 6px; border-bottom: 1px solid #CCA;'}).text.replace('\n  ', '').replace('.\n', '')
date

votes_df = pd.read_html('https://www.govtrack.us/congress/votes/117-2022/s134')[0].drop(columns='All Votes').rename(columns={'Unnamed: 0':'Votes', 'All Votes.1':'All Votes'})
votes_df

vote_result = soup.find('p', attrs={'style':'margin: 1em 0 0 0; font: 12px/20px serif;'}).text.replace('\n        ','').replace('\n',' ').replace('  Source: senate.gov.     ','').replace('MajorityRequired', 'Majority Required')
vote_result

pop_prop = soup.find('p', attrs={'style':'margin: 0; font: 12px/20px serif;'}).text.replace('\n                   ','').replace('\n               ','')
pop_prop

soup.find_all('table', attrs={'class':'vote-list stats'})

table = str(soup.find_all('table', attrs={'class':'vote-list stats'}))

states = re.findall('voter_group_1="([A-Z][a-z]+|[A-Z][a-z]+ [A-Z][a-z]+)" voter_group_2=', table)
print(len(states))
states

senators = re.findall('#444">([A-Z][a-z]+.+[A-Z][a-z]+)</a>', table)
print(len(senators))
senators

roll_call = re.findall('voter_group_0="([A-Z][a-z]{2})" voter_group_1', table)
print(len(roll_call))
roll_call

party = re.findall('voter_group_2="([A-Z][a-z]+)" voter_sort_1=', table)
print(len(party))
party

c117_2022_s134 = pd.DataFrame(senators, states).reset_index().rename(columns={'index':'States', 0:'Senators'})
c117_2022_s134['Roll Call'] = roll_call
c117_2022_s134['Party'] = party
c117_2022_s134

type(soup)

print(title)
print('------------------------------------------------------------------------------------------------------------')
print(date)
print('------------------------------------------------------------------------------------------------------------')
print(vote_result)
print('------------------------------------------------------------------------------------------------------------')
print(pop_prop)
display(votes_df)
display(c117_2022_s134)

c117_2022_s134['Decision'] = title
c117_2022_s134['Introduced'] = date
c117_2022_s134['Result'] = vote_result
c117_2022_s134['Proportion'] = pop_prop
c117_2022_s134

Pulling data from senate vote #133

response = requests.get('https://www.govtrack.us/congress/votes/117-2022/s133')

soup = BS(response.text)

title = soup.find('h1').text.replace('\n  ', '').replace('\n', '')

date = soup.find('div', attrs={'style':'margin: 2px 0 16px 0; font-size: 12px; font-weight: normal; color: black; line-height: 125%; padding-bottom: 6px; border-bottom: 1px solid #CCA;'}).text.replace('\n  ', '').replace('.\n', '')

votes_df = pd.read_html('https://www.govtrack.us/congress/votes/117-2022/s133')[0].drop(columns='All Votes').rename(columns={'Unnamed: 0':'Votes', 'All Votes.1':'All Votes'})

vote_result = soup.find('p', attrs={'style':'margin: 1em 0 0 0; font: 12px/20px serif;'}).text.replace('\n        ','').replace('\n',' ').replace('  Source: senate.gov.     ','').replace('MajorityRequired', 'Majority Required')

pop_prop = soup.find('p', attrs={'style':'margin: 0; font: 12px/20px serif;'}).text.replace('\n                   ','').replace('\n               ','')

soup.find_all('table', attrs={'class':'vote-list stats'})

table = str(soup.find_all('table', attrs={'class':'vote-list stats'}))

states = re.findall('voter_group_1="([A-Z][a-z]+|[A-Z][a-z]+ [A-Z][a-z]+)" voter_group_2=', table)

senators = re.findall('#444">([A-Z][a-z]+.+[A-Z][a-z]+)</a>', table)

roll_call = re.findall('voter_group_0="([A-Z][a-z]{2})" voter_group_1', table)

party = re.findall('voter_group_2="([A-Z][a-z]+)" voter_sort_1=', table)

c117_2022_s133 = pd.DataFrame(senators, states).reset_index().rename(columns={'index':'States', 0:'Senators'})
c117_2022_s133['Roll Call'] = roll_call
c117_2022_s133['Party'] = party
c117_2022_s133

print(title)
print('------------------------------------------------------------------------------------------------------------')
print(date)
print('------------------------------------------------------------------------------------------------------------')
print(vote_result)
print('------------------------------------------------------------------------------------------------------------')
print(pop_prop)
display(votes_df)
display(c117_2022_s133)

c117_2022_s133['Decision'] = title
c117_2022_s133['Introduced'] = date
c117_2022_s133['Result'] = vote_result
c117_2022_s133['Proportion'] = pop_prop
c117_2022_s133

Troubleshooting errors related to Vice President

response = requests.get('https://www.govtrack.us/congress/votes/117-2022/s115')

soup = BS(response.text)

print(soup.prettify())

soup.find_all('table', attrs={'class':'vote-list stats'})

table = str(soup.find_all('table', attrs={'class':'vote-list stats'}))

table.find('Kamala')

table[11500:12800]

states = re.findall('voter_group_1="([A-Z][a-z]+|[A-Z][a-z]+ [A-Z][a-z]+|The Vice President)" voter_group_2=', table)
print(len(states))
states

senators = re.findall('#444">([A-Z][a-z]+.+[A-Z][a-z]+)</a>', table)
print(len(senators))
senators

roll_call = re.findall('voter_group_0="([A-Z][a-z]{2})" voter_group_1', table)
print(len(roll_call))
roll_call

party = re.findall('voter_group_2="([A-Z][a-z]+|Vice President)" voter_sort_1=', table)
print(len(party))
party

c117_2022_s115 = pd.DataFrame(senators, states).reset_index().rename(columns={'index':'States', 0:'Senators'})
c117_2022_s115['Roll Call'] = roll_call
c117_2022_s115['Party'] = party
c117_2022_s115

print(title)
print('------------------------------------------------------------------------------------------------------------')
print(date)
print('------------------------------------------------------------------------------------------------------------')
print(vote_result)
print('------------------------------------------------------------------------------------------------------------')
print(pop_prop)
display(votes_df)
display(c117_2022_s134)

Troubleshooting errors related to roll_call

response = requests.get('https://www.govtrack.us/congress/votes/117-2022/s55')
soup = BS(response.text)
table = str(soup.find_all('table', attrs={'class':'vote-list stats'}))
soup.find_all('table', attrs={'class':'vote-list stats'})

re.findall('voter_group_0="(Yea|Nay|Not Voting|Present)" voter_group_1', table)

# User-defined function to test each senate vote #

def sen_vote_df(c, y, x, s_o_h='s'):
    URL = f'https://www.govtrack.us/congress/votes/{c}-{y}/{s_o_h}{x}'
    response = requests.get(URL)
    soup = BS(response.text)
    title = soup.find('h1').text.replace('\n  ', '').replace('\n', '')
    date = soup.find('div', attrs={'style':'margin: 2px 0 16px 0; font-size: 12px; font-weight: normal; color: black; line-height: 125%; padding-bottom: 6px; border-bottom: 1px solid #CCA;'}).text.replace('\n  ', '').replace('.\n', '')
    vote_result = soup.find('p', attrs={'style':'margin: 1em 0 0 0; font: 12px/20px serif;'}).text.replace('\n        ','').replace('\n',' ').replace('  Source: senate.gov.     ','').replace('MajorityRequired', 'Majority Required')
    pop_prop = soup.find('p', attrs={'style':'margin: 0; font: 12px/20px serif;'}).text.replace('\n                   ','').replace('\n               ','')
    table = str(soup.find_all('table', attrs={'class':'vote-list stats'}))
    states = re.findall('voter_group_1="([A-Z][a-z]+|[A-Z][a-z]+ [A-Z][a-z]+|The Vice President)" voter_group_2=', table)
    senators = re.findall('#444">([A-Z][a-z]+.+[A-Z][a-z]+)</a>', table)
    roll_call = re.findall('voter_group_0="(Yea|Nay|Not Voting|Present|Guilty|Not Guilty)" voter_group_1', table)
    party = re.findall('voter_group_2="([A-Z][a-z]+|Vice President)" voter_sort_1=', table)
    df = pd.DataFrame(senators, states).reset_index().rename(columns={'index':'States', 0:'Senators'})
    df['Roll Call'] = roll_call
    df['Party'] = party
    df['Decision'] = title
    df['Introduced'] = date
    df['Result'] = vote_result
    df['Proportion'] = pop_prop
    df.reset_index()
    display(df)

# Iterating through senate votes

Way to test code efficacy one year at a time

#con_years = ['100-1987','100-1988','101-1989','101-1990','102-1991','102-1992','103-1993','103-1994','104-1995','104-1996','105-1997','105-1998','106-1999','106-2000','107-2001','107-2002','108-2003','108-2004','109-2005','109-2006','110-2007','110-2008','111-2009','111-2010','112-2011','112-2012','113-2013','113-2014','114-2015','114-2016','115-2017','115-2018','116-2019','116-2020','117-2021','117-2022']
congress = '116'
year = '2020'

vote_no = 1
URL = f'https://www.govtrack.us/congress/votes/{congress}-{year}/s{vote_no}'
one_year_df = pd.DataFrame()

while requests.get(URL).status_code == 200:
    response = requests.get(URL)
    # code pertinent to individual page goes here v
    try:
        soup = BS(response.text)
        title = soup.find('h1').text.replace('\n  ', '').replace('\n', '')
        date = soup.find('div', attrs={'style':'margin: 2px 0 16px 0; font-size: 12px; font-weight: normal; color: black; line-height: 125%; padding-bottom: 6px; border-bottom: 1px solid #CCA;'}).text.replace('\n  ', '').replace('.\n', '')
        vote_result = soup.find('p', attrs={'style':'margin: 1em 0 0 0; font: 12px/20px serif;'}).text.replace('\n        ','').replace('\n',' ').replace('  Source: senate.gov.     ','').replace('MajorityRequired', 'Majority Required')
        pop_prop = soup.find('p', attrs={'style':'margin: 0; font: 12px/20px serif;'}).text.replace('\n                   ','').replace('\n               ','')
        table = str(soup.find_all('table', attrs={'class':'vote-list stats'}))
        states = re.findall('voter_group_1="([A-Z][a-z]+|[A-Z][a-z]+ [A-Z][a-z]+|The Vice President)" voter_group_2=', table)
        senators = re.findall('#444">([A-Z][a-z]+.+[A-Z][a-z]+)</a>', table)
        roll_call = re.findall('voter_group_0="(Yea|Nay|Not Voting|Present|Guilty|Not Guilty)" voter_group_1', table)
        party = re.findall('voter_group_2="([A-Z][a-z]+|Vice President)" voter_sort_1=', table)
        df = pd.DataFrame(senators, states).reset_index().rename(columns={'index':'States', 0:'Senators'})
        df['Roll Call'] = roll_call
        df['Party'] = party
        df['Decision'] = title
        df['Introduced'] = date
        df['Result'] = vote_result
        df['Proportion'] = pop_prop
        one_year_df = pd.concat([one_year_df, df], axis=0)
        print(f'SUCCESS {vote_no}')
    except:
        print(f'Operation unsuccessful on senate vote {vote_no}')
    # code pertinent to individual page goes here ^
    vote_no += 1
    URL = f'https://www.govtrack.us/congress/votes/{congress}-{year}/s{vote_no}'
    
one_year_df

Getting every roll call at once from multiple congresses/years

In [ ]:
con_years = ['112-2012','113-2013','113-2014','114-2015','114-2016','115-2017','115-2018','116-2019','116-2020','117-2021','117-2022']
filter_df = pd.DataFrame()
for con_year in tqdm(con_years):
    vote_no = 1
    URL = f'https://www.govtrack.us/congress/votes/{con_year}/s{vote_no}'
    while requests.get(URL).status_code == 200:
        response = requests.get(URL)
        # code pertinent to individual page goes here v
        try:
            soup = BS(response.text)
            title = soup.find('h1').text.replace('\n  ', '').replace('\n', '')
            date = soup.find('div', attrs={'style':'margin: 2px 0 16px 0; font-size: 12px; font-weight: normal; color: black; line-height: 125%; padding-bottom: 6px; border-bottom: 1px solid #CCA;'}).text.replace('\n  ', '').replace('.\n', '')
            vote_result = soup.find('p', attrs={'style':'margin: 1em 0 0 0; font: 12px/20px serif;'}).text.replace('\n        ','').replace('\n',' ').replace('  Source: senate.gov.     ','').replace('MajorityRequired', 'Majority Required')
            pop_prop = soup.find('p', attrs={'style':'margin: 0; font: 12px/20px serif;'}).text.replace('\n                   ','').replace('\n               ','')
            table = str(soup.find_all('table', attrs={'class':'vote-list stats'}))
            states = re.findall('voter_group_1="([A-Z][a-z]+|[A-Z][a-z]+ [A-Z][a-z]+|The Vice President)" voter_group_2=', table)
            senators = re.findall('#444">([A-Z][a-z]+.+[A-Z][a-z]+)</a>', table)
            roll_call = re.findall('voter_group_0="(Yea|Nay|Not Voting|Present|Guilty|Not Guilty)" voter_group_1', table)
            party = re.findall('voter_group_2="([A-Z][a-z]+|Vice President)" voter_sort_1=', table)
            df = pd.DataFrame(senators, states).reset_index().rename(columns={'index':'States', 0:'Senators'})
            df['Roll Call'] = roll_call
            df['Party'] = party
            df['Decision'] = title
            df['Introduced'] = date
            df['Result'] = vote_result
            df['Proportion'] = pop_prop
            df['con_year'] = f'{con_year}'
            filter_df = pd.concat([filter_df, df], axis=0)
        except:
            print(f'Unsuccessful on {con_year}, senate vote #{vote_no}')
        # code pertinent to individual page goes here ^
        vote_no += 1
        URL = f'https://www.govtrack.us/congress/votes/{con_year}/s{vote_no}'
    print(f'{con_year}: DONE')

# Roll Call EDA

filter_df = pd.read_csv('data/voting_records.csv')

In [1559]:
filter_df

,States,Senators,Roll Call,Party,Decision,Introduced,Result,Proportion,Congress and Year
0,Hawaii,"Akaka, Daniel",Yea,Democrat,"On the Nomination PN468: John M. Gerrard, of N...","Jan 23, 2012 at 5:32 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 78% of the country’s...,112-2012
1,Montana,"Baucus, Max",Yea,Democrat,"On the Nomination PN468: John M. Gerrard, of N...","Jan 23, 2012 at 5:32 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 78% of the country’s...,112-2012
2,Alaska,"Begich, Mark",Yea,Democrat,"On the Nomination PN468: John M. Gerrard, of N...","Jan 23, 2012 at 5:32 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 78% of the country’s...,112-2012
3,Colorado,"Bennet, Michael",Yea,Democrat,"On the Nomination PN468: John M. Gerrard, of N...","Jan 23, 2012 at 5:32 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 78% of the country’s...,112-2012
4,New Mexico,"Bingaman, Jeff",Yea,Democrat,"On the Nomination PN468: John M. Gerrard, of N...","Jan 23, 2012 at 5:32 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 78% of the country’s...,112-2012
...,...,...,...,...,...,...,...,...,...
339005,North Carolina,"Tillis, Thom",Nay,Republican,On the Nomination PN1783: Ketanji Brown Jackso...,"Apr 7, 2022 at 1:53 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 57% of the country’s...,117-2022
339006,Pennsylvania,"Toomey, Pat",Nay,Republican,On the Nomination PN1783: Ketanji Brown Jackso...,"Apr 7, 2022 at 1:53 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 57% of the country’s...,117-2022
339007,Alabama,"Tuberville, Tommy",Nay,Republican,On the Nomination PN1783: Ketanji Brown Jackso...,"Apr 7, 2022 at 1:53 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 57% of the country’s...,117-2022
339008,Mississippi,"Wicker, Roger",Nay,Republican,On the Nomination PN1783: Ketanji Brown Jackso...,"Apr 7, 2022 at 1:53 p.m. ET",Nomination Confirmed. Simple Majority Required.,The Yea votes represented 57% of the country’s...,117-2022


In [1560]:
def leg_list(x):
    return list(re.findall('S. \d+|H.R. \d+|S.Res. \d+|H.Res. \d+|S.Con.Res. \d+|H.Con.Res. \d+|S.J.Res. \d+|H.J.Res. \d+', x))
filter_df['Relevant Bills'] = filter_df['Decision'].apply(leg_list)

In [1561]:
filter_df = filter_df[filter_df['Relevant Bills'].apply(lambda x: len(x) != 0)].reset_index().drop(columns=['index'])
filter_df

,States,Senators,Roll Call,Party,Decision,Introduced,Result,Proportion,Congress and Year,Relevant Bills
0,West Virginia,"Manchin, Joe",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res. 98]
1,Nebraska,"Nelson, Ben",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res. 98]
2,Tennessee,"Alexander, Lamar",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res. 98]
3,New Hampshire,"Ayotte, Kelly",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res. 98]
4,Wyoming,"Barrasso, John",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res. 98]
...,...,...,...,...,...,...,...,...,...,...
150296,North Carolina,"Tillis, Thom",Yea,Republican,H.R. 6968: Ending Importation of Russian Oil Act,"Apr 7, 2022 at 10:58 a.m. ET. On Passage of t...",Bill Passed. Simple Majority Required.,The Yea votes represented 100% of the country’...,117-2022,[H.R. 6968]
150297,Pennsylvania,"Toomey, Pat",Yea,Republican,H.R. 6968: Ending Importation of Russian Oil Act,"Apr 7, 2022 at 10:58 a.m. ET. On Passage of t...",Bill Passed. Simple Majority Required.,The Yea votes represented 100% of the country’...,117-2022,[H.R. 6968]
150298,Alabama,"Tuberville, Tommy",Yea,Republican,H.R. 6968: Ending Importation of Russian Oil Act,"Apr 7, 2022 at 10:58 a.m. ET. On Passage of t...",Bill Passed. Simple Majority Required.,The Yea votes represented 100% of the country’...,117-2022,[H.R. 6968]
150299,Mississippi,"Wicker, Roger",Yea,Republican,H.R. 6968: Ending Importation of Russian Oil Act,"Apr 7, 2022 at 10:58 a.m. ET. On Passage of t...",Bill Passed. Simple Majority Required.,The Yea votes represented 100% of the country’...,117-2022,[H.R. 6968]


In [1562]:
for index,row in tqdm(filter_df.iterrows()):
    list_len = len(filter_df.loc[index, 'Relevant Bills'])
    for x in range(list_len):
        bill_list = filter_df.loc[index, 'Relevant Bills']
        bill_list[x] = bill_list[x].replace(' ','')

0it [00:00, ?it/s]

In [1563]:
filter_df

,States,Senators,Roll Call,Party,Decision,Introduced,Result,Proportion,Congress and Year,Relevant Bills
0,West Virginia,"Manchin, Joe",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res.98]
1,Nebraska,"Nelson, Ben",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res.98]
2,Tennessee,"Alexander, Lamar",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res.98]
3,New Hampshire,"Ayotte, Kelly",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res.98]
4,Wyoming,"Barrasso, John",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[H.J.Res.98]
...,...,...,...,...,...,...,...,...,...,...
150296,North Carolina,"Tillis, Thom",Yea,Republican,H.R. 6968: Ending Importation of Russian Oil Act,"Apr 7, 2022 at 10:58 a.m. ET. On Passage of t...",Bill Passed. Simple Majority Required.,The Yea votes represented 100% of the country’...,117-2022,[H.R.6968]
150297,Pennsylvania,"Toomey, Pat",Yea,Republican,H.R. 6968: Ending Importation of Russian Oil Act,"Apr 7, 2022 at 10:58 a.m. ET. On Passage of t...",Bill Passed. Simple Majority Required.,The Yea votes represented 100% of the country’...,117-2022,[H.R.6968]
150298,Alabama,"Tuberville, Tommy",Yea,Republican,H.R. 6968: Ending Importation of Russian Oil Act,"Apr 7, 2022 at 10:58 a.m. ET. On Passage of t...",Bill Passed. Simple Majority Required.,The Yea votes represented 100% of the country’...,117-2022,[H.R.6968]
150299,Mississippi,"Wicker, Roger",Yea,Republican,H.R. 6968: Ending Importation of Russian Oil Act,"Apr 7, 2022 at 10:58 a.m. ET. On Passage of t...",Bill Passed. Simple Majority Required.,The Yea votes represented 100% of the country’...,117-2022,[H.R.6968]


In [1564]:
for index,value in tqdm(filter_df['Congress and Year'].iteritems()):
    if value == '112-2012':
        filter_df.loc[index, 'Congress'] = r'112th Congress (2011-2012)'
    if (value == '113-2013') | (value == '113-2014'):
        filter_df.loc[index, 'Congress'] = r'113th Congress (2013-2014)'
    if (value == '114-2015') | (value == '114-2016'):
        filter_df.loc[index, 'Congress'] = r'114th Congress (2015-2016)'
    if (value == '115-2017') | (value == '115-2018'):
        filter_df.loc[index, 'Congress'] = r'115th Congress (2017-2018)'
    if (value == '116-2019') | (value == '116-2020'):
        filter_df.loc[index, 'Congress'] = r'116th Congress (2019-2020)'
    if (value == '117-2021') | (value == '117-2022'):
        filter_df.loc[index, 'Congress'] = r'117th Congress (2021-2022)'

0it [00:00, ?it/s]

In [1565]:
for index,row in tqdm(filter_df.iterrows()):
    list_len = len(filter_df.loc[index, 'Relevant Bills'])
    for x in range(list_len):
        bill_list = filter_df.loc[index, 'Relevant Bills']
        bill_list[x] = {filter_df.loc[index, 'Congress']:bill_list[x]}

0it [00:00, ?it/s]

In [1566]:
for index,value in tqdm(filter_df.iterrows()):
    list_len = len(filter_df.loc[index, 'Relevant Bills'])
    for x in range(list_len):
        bill_list = filter_df.loc[index, 'Relevant Bills']
        if bill_list[x] in d['Health']:
            filter_df.loc[index, 'Policy Area'] = 'Health'
        if bill_list[x] in d['Public+Lands+and+Natural+Resources']:
            filter_df.loc[index, 'Policy Area'] = 'Public Lands and Natural Resources'
        if bill_list[x] in d['Agriculture+and+Food']:
            filter_df.loc[index, 'Policy Area'] = 'Agriculture and Food'
        if bill_list[x] in d['Economics+and+Public+Finance']:
            filter_df.loc[index, 'Policy Area'] = 'Economics and Public Finance'
        if bill_list[x] in d['Housing+and+Community+Development']:
            filter_df.loc[index, 'Policy Area'] = 'Housing and Community Development'
        if bill_list[x] in d['Water+Resources+Development']:
            filter_df.loc[index, 'Policy Area'] = 'Water Resources Development'

0it [00:00, ?it/s]

In [1567]:
filter_df = filter_df[filter_df['Policy Area'].notna()]
filter_df

,States,Senators,Roll Call,Party,Decision,Introduced,Result,Proportion,Congress and Year,Relevant Bills,Congress,Policy Area
0,West Virginia,"Manchin, Joe",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
1,Nebraska,"Nelson, Ben",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
2,Tennessee,"Alexander, Lamar",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
3,New Hampshire,"Ayotte, Kelly",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
4,Wyoming,"Barrasso, John",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
...,...,...,...,...,...,...,...,...,...,...,...,...
149596,Vermont,"Sanders, Bernie",Nay,Independent,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117-2022,[{'117th Congress (2021-2022)': 'S.J.Res.37'}],117th Congress (2021-2022),Health
149597,Utah,"Romney, Mitt",Nay,Republican,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117-2022,[{'117th Congress (2021-2022)': 'S.J.Res.37'}],117th Congress (2021-2022),Health
149598,Illinois,"Duckworth, Tammy",Not Voting,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117-2022,[{'117th Congress (2021-2022)': 'S.J.Res.37'}],117th Congress (2021-2022),Health
149599,New Jersey,"Menendez, Bob",Not Voting,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117-2022,[{'117th Congress (2021-2022)': 'S.J.Res.37'}],117th Congress (2021-2022),Health


In [1568]:
filter_df = filter_df[filter_df['Relevant Bills'].apply(lambda x: len(x) == 1)]
filter_df

,States,Senators,Roll Call,Party,Decision,Introduced,Result,Proportion,Congress and Year,Relevant Bills,Congress,Policy Area
0,West Virginia,"Manchin, Joe",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
1,Nebraska,"Nelson, Ben",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
2,Tennessee,"Alexander, Lamar",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
3,New Hampshire,"Ayotte, Kelly",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
4,Wyoming,"Barrasso, John",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112-2012,[{'112th Congress (2011-2012)': 'H.J.Res.98'}],112th Congress (2011-2012),Economics and Public Finance
...,...,...,...,...,...,...,...,...,...,...,...,...
149596,Vermont,"Sanders, Bernie",Nay,Independent,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117-2022,[{'117th Congress (2021-2022)': 'S.J.Res.37'}],117th Congress (2021-2022),Health
149597,Utah,"Romney, Mitt",Nay,Republican,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117-2022,[{'117th Congress (2021-2022)': 'S.J.Res.37'}],117th Congress (2021-2022),Health
149598,Illinois,"Duckworth, Tammy",Not Voting,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117-2022,[{'117th Congress (2021-2022)': 'S.J.Res.37'}],117th Congress (2021-2022),Health
149599,New Jersey,"Menendez, Bob",Not Voting,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117-2022,[{'117th Congress (2021-2022)': 'S.J.Res.37'}],117th Congress (2021-2022),Health


In [1569]:
filter_df['Policy Area'].unique()

array(['Economics and Public Finance', 'Health', 'Agriculture and Food',
       'Public Lands and Natural Resources',
       'Water Resources Development'], dtype=object)

In [1570]:
for index,value in tqdm(filter_df['Policy Area'].iteritems()):
    if value == 'Health':
        filter_df.loc[index, 'Sector'] = 'Healthcare'
    if value == 'Public Lands and Natural Resources':
        filter_df.loc[index, 'Sector'] = 'Basic Materials'
    if value == 'Agriculture and Food':
        filter_df.loc[index, 'Sector'] = 'Consumer Defensive'
    if value == 'Economics and Public Finance':
        filter_df.loc[index, 'Sector'] = 'Financial Services'
    if value == 'Housing and Community Development':
        filter_df.loc[index, 'Sector'] = 'Industrials'
    if value == 'Water Resources Development':
        filter_df.loc[index, 'Sector'] = 'Utilities'

0it [00:00, ?it/s]

C:\Users\cocat\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\cocat\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [1573]:
votes = filter_df.reset_index().drop(columns=['index', 'Congress and Year', 'Relevant Bills'])
votes

,States,Senators,Roll Call,Party,Decision,Introduced,Result,Proportion,Congress,Policy Area,Sector
0,West Virginia,"Manchin, Joe",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services
1,Nebraska,"Nelson, Ben",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services
2,Tennessee,"Alexander, Lamar",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services
3,New Hampshire,"Ayotte, Kelly",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services
4,Wyoming,"Barrasso, John",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services
...,...,...,...,...,...,...,...,...,...,...,...
68199,Vermont,"Sanders, Bernie",Nay,Independent,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare
68200,Utah,"Romney, Mitt",Nay,Republican,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare
68201,Illinois,"Duckworth, Tammy",Not Voting,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare
68202,New Jersey,"Menendez, Bob",Not Voting,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare


# Stock/Senator Merge

In [1574]:
def leg_list(x):
    return re.search('S. \d+|H.R. \d+|S.Res. \d+|H.Res. \d+|S.Con.Res. \d+|H.Con.Res. \d+|S.J.Res. \d+|H.J.Res. \d+', x).group(0)
votes['Relevant Bill'] = votes['Decision'].apply(leg_list)

In [1575]:
votes['Relevant Stocks Owned'] = [list() for x in range(len(votes.index))]

In [1576]:
votes

,States,Senators,Roll Call,Party,Decision,Introduced,Result,Proportion,Congress,Policy Area,Sector,Relevant Bill,Relevant Stocks Owned
0,West Virginia,"Manchin, Joe",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,[]
1,Nebraska,"Nelson, Ben",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,[]
2,Tennessee,"Alexander, Lamar",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,[]
3,New Hampshire,"Ayotte, Kelly",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,[]
4,Wyoming,"Barrasso, John",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68199,Vermont,"Sanders, Bernie",Nay,Independent,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare,S.J.Res. 37,[]
68200,Utah,"Romney, Mitt",Nay,Republican,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare,S.J.Res. 37,[]
68201,Illinois,"Duckworth, Tammy",Not Voting,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare,S.J.Res. 37,[]
68202,New Jersey,"Menendez, Bob",Not Voting,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare,S.J.Res. 37,[]


In [1577]:
for x in tqdm(range(0, len(votes.index)-1)):
    if votes.loc[x, 'Senators'] in sen_list:
        votes.loc[x, 'Relevant Stocks Owned'].extend(list(set(sector_dict[votes.loc[x]['Sector']]) & set(sen_stock_dict[votes.loc[x]['Senators']])))

  0%|          | 0/68203 [00:00<?, ?it/s]

In [1580]:
votes = votes[votes['Relevant Stocks Owned'].apply(lambda x: len(x) != 0)].reset_index().drop(columns='index')

In [1583]:
votes

,States,Senators,Roll Call,Party,Decision,Introduced,Result,Proportion,Congress,Policy Area,Sector,Relevant Bill,Relevant Stocks Owned
0,West Virginia,"Manchin, Joe",Yea,Democrat,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,[HBAN]
1,Maine,"Collins, Susan",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,"[CFR, AXP, PX, AON, JPM, USB, WFC]"
2,Wyoming,"Enzi, Michael",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,[KEY]
3,North Dakota,"Hoeven, John",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,"[MET, WTW, USB, WFC, BX, IVZ, PRU]"
4,Oklahoma,"Inhofe, Jim",Yea,Republican,On the Motion to Proceed H.J.Res. 98,"Jan 26, 2012 at 11:57 a.m. ET",Motion to Proceed Rejected. Simple Majority Re...,The Nay votes represented 58% of the country’s...,112th Congress (2011-2012),Economics and Public Finance,Financial Services,H.J.Res. 98,"[V, BLK, PYPL, C, BAM, WFC, SYF, AIG, COF, DFS]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12733,Maryland,"Van Hollen, Chris",Nay,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare,S.J.Res. 37,[JNJ]
12734,Virginia,"Warner, Mark",Nay,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare,S.J.Res. 37,[HALO]
12735,Rhode Island,"Whitehouse, Sheldon",Nay,Democrat,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare,S.J.Res. 37,"[CERN, MCK, SYK, BMY, ABT, PFE, ILMN, BIIB, IO..."
12736,Maine,"King, Angus",Nay,Independent,S.J.Res. 37: A joint resolution providing for ...,"Mar 15, 2022 at 5:30 p.m. ET. On the Joint Re...",Joint Resolution Passed. Simple Majority Requi...,The Yea votes represented 48% of the country’s...,117th Congress (2021-2022),Health,Healthcare,S.J.Res. 37,"[WAT, TEVA, NVS, GILD, TMO]"
